<a href="https://colab.research.google.com/github/astuteshu/stock-price-prediction/blob/master/stock_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#using LSTM to predict the closing stock price of a corporation by observing the past 60 days stock prize 

In [ ]:
 #import libraries
 import math
 import pandas_datareader as web
 import numpy as np
 import pandas as pd
 from sklearn.preprocessing import MinMaxScaler
 from keras.models import Sequential
 from keras.layers import Dense,LSTM
 import matplotlib.pyplot as plt
 plt.style.use('fivethirtyeight')

In [ ]:
#get stock prices
df = web.DataReader('AAPL',data_source='yahoo',start='2012-01-01',end='2019-12-17')
#show data
df
df.shape

In [ ]:
#plot the closing price data
plt.figure(figsize=(16,8))
plt.title('closing price history')
plt.plot(df['Close'])
plt.xlabel('date',fontsize=18)
plt.ylabel('closing price in $',fontsize=18)
plt.show()

In [ ]:
#newdataframe with only closing price
data=df.filter(['Close'])
#convert dataframe to numpy array
dataset=data.values
#get the number of rows to train the model on
training_data_len=math.ceil(len(dataset)*.8)#as we want to train the data only on ceil of 80% of the dataset
training_data_len#number of rows

In [ ]:
#scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
  #create the training dataset
  #create scaled training data set
  train_data= scaled_data[0:training_data_len,:]
  #split the data into x_train and y_train datasets
  x_train=[]    
  y_train=[]
  for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0])
    y_train.append(train_data[i,0])
    if i<=61:
      print(x_train)
      print(y_train)
      print()

In [ ]:
#convert x_train & y_train to numpy array to use lstm on them
x_train, y_train = np.array(x_train), np.array(y_train) 

In [ ]:
 #reshape the x_train data
 #why as lstm expects dataset to be 3d in terms of no of timesteps no of samples no of features
 # and currently dataset is 2d
 # in reshaped 3d dataset we have no.of samples as no.of rows of 2d array
 # no. of timesteps is no. of columns 
 #no. of features is closing price i.e 1
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1)) 
x_train.shape

In [ ]:
 #build lstm model
model =Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))



In [ ]:
#compile the model
model.compile(optimizer='adam',loss='mean_squared_error')


In [ ]:
 #train the model
model.fit(x_train, y_train, batch_size=1, epochs=1) 

In [ ]:
#create testing dataset
#create new array having scaled values from idx 1543 to 2003
test_data = scaled_data[training_data_len-60:,:]
#create datasets
x_test = []
y_test = dataset[training_data_len:,:]
for i in range(60,len(test_data)):
  x_test.append(test_data[i-60:i,0])
  
   

In [ ]:
#convert data to numpy array
x_test = np.array(x_test)

In [ ]:
#reshape data
x_test = np.reshape(x_test,(x_test.shape[0], x_test.shape[1],1))

In [ ]:
#get the models predicted price value for x_test dataset
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
#evaluate model by getting root mean squared error
#low value of rmse gives better fit
rmse=np.sqrt(np.mean(predictions-y_test)**2)
rmse

In [ ]:
#plot the data
train =data[:training_data_len]
valid =data[training_data_len:]
valid['Predictions']=predictions
#visualize
plt.figure(figsize=(16,8))
plt.title('model')
plt.xlabel('date',fontsize=18)
plt.ylabel('colsing price in $',fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close','Predictions']])
plt.legend(['Train','Val','Predictions'],loc='lower right')
plt.show()

In [ ]:
#show the valid and predicted prices
valid

In [ ]:
#to predict stock price for a given day
apple_quote=web.DataReader('AAPL',data_source='yahoo',start='2012-01-01',end='2019-12-17')
new_df=apple_quote.filter(['Close'])
last_60_days=new_df[-60:].values
last_60_days_scaled=scaler.transform(last_60_days)
X_test=[]
X_test.append(last_60_days_scaled)
X_test=np.array(X_test)
X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
pred_price=model.predict(X_test)
pred_price=scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
apple_quote_2=web.DataReader('AAPL',data_source='yahoo',start='2019-12-18',end='2019-12-18' )
print(apple_quote_2['Close'])